In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
# 노래 분위기와 주제는 이미 label encoding을 진행한 상태
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/soyo_project/join_table.csv')
df.head()

,Unnamed: 0,program_name,singer,music,views,likes,comments,atmosphere,subject,atmos_label,sub_label
0,0,미스터트롯,임영웅,바램,2223366,22906,4689,슬픔,인생,1,4
1,1,미스터트롯,영탁,사내,604639,6583,1599,활기참,인생,6,4
2,2,미스터트롯,영탁,댄싱퀸,828628,7804,1884,신남,남녀사랑,2,2
3,3,미스터트롯,임영웅,댄싱퀸,828628,7804,1884,신남,남녀사랑,2,2
4,4,미스터트롯,영탁,막걸리 한 잔,1805978,19675,7231,활기참,부모효도,6,3


In [ ]:
df_pre = df.drop(['Unnamed: 0', 'atmosphere', 'subject'], axis=1)
df_pre.head()

,program_name,singer,music,views,likes,comments,atmos_label,sub_label
0,미스터트롯,임영웅,바램,2223366,22906,4689,1,4
1,미스터트롯,영탁,사내,604639,6583,1599,6,4
2,미스터트롯,영탁,댄싱퀸,828628,7804,1884,2,2
3,미스터트롯,임영웅,댄싱퀸,828628,7804,1884,2,2
4,미스터트롯,영탁,막걸리 한 잔,1805978,19675,7231,6,3


In [ ]:
# 원-핫 인코딩 진행
df_pre = pd.get_dummies(data=df_pre, columns=['atmos_label'], prefix='atmosphere')
df_pre = pd.get_dummies(data=df_pre, columns=['sub_label'], prefix='subject')
df_pre.head()

,program_name,singer,music,views,likes,comments,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5
0,미스터트롯,임영웅,바램,2223366,22906,4689,0,1,0,0,0,0,0,0,0,0,0,1,0
1,미스터트롯,영탁,사내,604639,6583,1599,0,0,0,0,0,0,1,0,0,0,0,1,0
2,미스터트롯,영탁,댄싱퀸,828628,7804,1884,0,0,1,0,0,0,0,0,0,1,0,0,0
3,미스터트롯,임영웅,댄싱퀸,828628,7804,1884,0,0,1,0,0,0,0,0,0,1,0,0,0
4,미스터트롯,영탁,막걸리 한 잔,1805978,19675,7231,0,0,0,0,0,0,1,0,0,0,1,0,0


In [ ]:
# 각 프로그램마다 조회수, 좋아요수, 댓글수를 합쳐서 비율 컬럼 만들기 -> 프로그램 내 인기 척도가 될 수 있음
# 절대적인 숫자로 판단하면 미스터트롯 임영웅 수치가 너무 크고, 미스트롯2의 경우 동영상이 막 업로드되기 때문에 상대적으로 적음 
program_list = list(df_pre['program_name'].unique())

for i in program_list:
  sum_n = sum(df_pre.loc[df_pre['program_name'] == i, 'views'].values)
  df_pre.loc[df_pre['program_name'] == i, 'views_per'] = df_pre.loc[df_pre['program_name'] == i, 'views'].apply(lambda x: x/sum_n)

  sum_n = sum(df_pre.loc[df_pre['program_name'] == i, 'likes'].values)
  df_pre.loc[df_pre['program_name'] == i, 'likes_per'] = df_pre.loc[df_pre['program_name'] == i, 'likes'].apply(lambda x: x/sum_n)

  sum_n = sum(df_pre.loc[df_pre['program_name'] == i, 'comments'].values)
  df_pre.loc[df_pre['program_name'] == i, 'comments_per'] = df_pre.loc[df_pre['program_name'] == i, 'comments'].apply(lambda x: x/sum_n)


df_pre.head()

,program_name,singer,music,views,likes,comments,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5,views_per,likes_per,comments_per
0,미스터트롯,임영웅,바램,2223366,22906,4689,0,1,0,0,0,0,0,0,0,0,0,1,0,0.035656,0.029955,0.006519
1,미스터트롯,영탁,사내,604639,6583,1599,0,0,0,0,0,0,1,0,0,0,0,1,0,0.009697,0.008609,0.002223
2,미스터트롯,영탁,댄싱퀸,828628,7804,1884,0,0,1,0,0,0,0,0,0,1,0,0,0,0.013289,0.010205,0.002619
3,미스터트롯,임영웅,댄싱퀸,828628,7804,1884,0,0,1,0,0,0,0,0,0,1,0,0,0,0.013289,0.010205,0.002619
4,미스터트롯,영탁,막걸리 한 잔,1805978,19675,7231,0,0,0,0,0,0,1,0,0,0,1,0,0,0.028962,0.025729,0.010054


In [ ]:
df_train = df_pre[df_pre['program_name'] != '미스트롯2'] # 훈련셋
df_test = df_pre[df_pre['program_name'] == '미스트롯2'] # 테스트셋
print(df_test.shape)
df_test.head()

(77, 22)


,program_name,singer,music,views,likes,comments,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5,views_per,likes_per,comments_per
358,미스트롯2,양지은,아버지와 딸,112756,1398,327,0,0,0,0,1,0,0,0,0,0,1,0,0,0.005656,0.007407,0.002744
359,미스트롯2,양지은,도로남,94724,342,184,0,0,0,1,0,0,0,0,0,0,0,1,0,0.004751,0.001812,0.001544
360,미스트롯2,양지은,빈대떡 신사,367588,2466,580,0,0,0,0,0,0,1,0,0,0,0,1,0,0.018437,0.013066,0.004867
361,미스트롯2,양지은,왕서방,120975,811,723,0,0,0,1,0,0,0,0,0,1,0,0,0,0.006068,0.004297,0.006067
362,미스트롯2,양지은,부초같은 인생,133631,732,474,0,0,0,0,0,0,1,0,0,0,0,1,0,0.006703,0.003879,0.003978


In [ ]:
# 진선미 리스트 만들기
first_lst = ['임영웅', '영탁', '이찬원', '송가인', '정미애', '홍자', '박세욱', '김다현', '조문근', '강문경', '나상도', '한봄', '안성준', '김소연', '김재롱']
def jinsonmi(x):
  if x in first_lst:
    return 1
  else:
    return 0

In [ ]:
df_train['target'] = df_train['singer'].apply(lambda x: jinsonmi(x))
df_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,program_name,singer,music,views,likes,comments,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5,views_per,likes_per,comments_per,target
0,미스터트롯,임영웅,바램,2223366,22906,4689,0,1,0,0,0,0,0,0,0,0,0,1,0,0.035656,0.029955,0.006519,1
1,미스터트롯,영탁,사내,604639,6583,1599,0,0,0,0,0,0,1,0,0,0,0,1,0,0.009697,0.008609,0.002223,1
2,미스터트롯,영탁,댄싱퀸,828628,7804,1884,0,0,1,0,0,0,0,0,0,1,0,0,0,0.013289,0.010205,0.002619,1
3,미스터트롯,임영웅,댄싱퀸,828628,7804,1884,0,0,1,0,0,0,0,0,0,1,0,0,0,0.013289,0.010205,0.002619,1
4,미스터트롯,영탁,막걸리 한 잔,1805978,19675,7231,0,0,0,0,0,0,1,0,0,0,1,0,0,0.028962,0.025729,0.010054,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,트로트의 민족,김민건,오르락내리락,6092,81,25,0,0,0,0,0,0,1,0,0,0,0,1,0,0.005669,0.008012,0.009273,0
354,트로트의 민족,김민건,다함께 차차차,6092,81,25,0,0,1,0,0,0,0,0,0,1,0,0,0,0.005669,0.008012,0.009273,0
355,트로트의 민족,김민건,사랑님,8458,72,8,0,0,0,0,0,0,1,0,0,1,0,0,0,0.007870,0.007122,0.002967,0
356,트로트의 민족,김민건,돌리도,5935,55,9,0,0,0,0,0,0,1,0,0,1,0,0,0,0.005522,0.005440,0.003338,0


In [ ]:
train_train = df_train.iloc[:, 6:-1] # 퍼센트를 학습에 사용
train_target = df_train.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from sklearn.preprocessing import StandardScaler

In [ ]:
# train_train

,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5,views_per,likes_per,comments_per
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.035656,0.029955,0.006519
1,0,0,0,0,0,0,1,0,0,0,0,1,0,0.009697,0.008609,0.002223
2,0,0,1,0,0,0,0,0,0,1,0,0,0,0.013289,0.010205,0.002619
3,0,0,1,0,0,0,0,0,0,1,0,0,0,0.013289,0.010205,0.002619
4,0,0,0,0,0,0,1,0,0,0,1,0,0,0.028962,0.025729,0.010054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,0,0,0,0,0,0,1,0,0,0,0,1,0,0.005669,0.008012,0.009273
354,0,0,1,0,0,0,0,0,0,1,0,0,0,0.005669,0.008012,0.009273
355,0,0,0,0,0,0,1,0,0,1,0,0,0,0.007870,0.007122,0.002967
356,0,0,0,0,0,0,1,0,0,1,0,0,0,0.005522,0.005440,0.003338


In [ ]:
# 훈련셋 스케일링 진행
scaled_train = train_train
scaler = StandardScaler()
scaler.fit(train_train[['views_per', 'likes_per', 'comments_per']])
scaled_train.iloc[:, -3:] = scaler.transform(train_train[['views_per', 'likes_per', 'comments_per']])
X_train = scaled_train
X_train.head()

,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5,views_per,likes_per,comments_per
0,0,1,0,0,0,0,0,0,0,0,0,1,0,1.320160,0.894594,-0.391271
1,0,0,0,0,0,0,1,0,0,0,0,1,0,-0.259893,-0.299786,-0.616997
2,0,0,1,0,0,0,0,0,0,1,0,0,0,-0.041256,-0.210444,-0.596178
3,0,0,1,0,0,0,0,0,0,1,0,0,0,-0.041256,-0.210444,-0.596178
4,0,0,0,0,0,0,1,0,0,0,1,0,0,0.912744,0.658177,-0.205576


In [ ]:
# 테스트셋 스케일링 진행
scaled_test = df_test.iloc[:, 6:]
scaled_test.iloc[:, -3:] = scaler.transform(scaled_test[['views_per', 'likes_per', 'comments_per']])
X_test = scaled_test
X_test.head()

,atmosphere_0,atmosphere_1,atmosphere_2,atmosphere_3,atmosphere_4,atmosphere_5,atmosphere_6,subject_0,subject_1,subject_2,subject_3,subject_4,subject_5,views_per,likes_per,comments_per
358,0,0,0,0,1,0,0,0,0,0,1,0,0,-0.505851,-0.367008,-0.589624
359,0,0,0,1,0,0,0,0,0,0,0,1,0,-0.560901,-0.680082,-0.652676
360,0,0,0,0,0,0,1,0,0,0,0,1,0,0.272133,-0.050377,-0.478072
361,0,0,0,1,0,0,0,0,0,1,0,0,0,-0.480759,-0.541037,-0.415020
362,0,0,0,0,0,0,1,0,0,0,0,1,0,-0.442121,-0.564458,-0.524809


In [ ]:
# 아래에서 이미 pycaret을 돌렸고, gbc를 주모델로 사용하기로 결정했기 때문에 그대로 사용
# gridsearch 최적화 파라미터: {'criterion': 'mae', 'learning_rate': 0.01, 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 50, 'subsample': 0.7}

from sklearn.ensemble import GradientBoostingClassifier
gbc_clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.01, max_features='log2',
      criterion='mae', max_depth=6, subsample=0.7, random_state=0)
gbc_clf.fit(X_train, train_target)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='mae', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=6,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
# cross validation 결과 평균 65%의 정확도
cross_val_score(gbc_clf, X_train, train_target, cv=3, scoring='accuracy')

array([0.63333333, 0.66386555, 0.63865546])

In [ ]:
prediction = gbc_clf.predict(X_test) # 모델 예측
can_list = list(np.where(prediction == 1))
can_list[0]

array([ 6,  9, 18, 23, 32, 33, 37])

In [ ]:
# 진선미라고 예측한 후보들은 총 4명
for i in can_list[0]:
  print(df_test.iloc[i, 1])

양지은
홍지윤
홍지윤
김다현
김태연
김태연
김태연


## 진선미 후보는 홍지윤, 김태연, 김다현
* 진선미 순서도 굳이 예측해본다면 진선미라고 예측한 노래가 가장 많은 순서대로 기준을 잡았을 때 "진 -> 김태연 / 선 -> 홍지윤 / 미 -> 김다현"

In [ ]:
# 진선미 후보가 4명이 나왔기 때문에 어떤 노래를 가지고 진선미라고 예측했는지 살펴보기
# 가수들이 부른 노래에 대해 진선미라고 예측한 노래가 가장 많은 순서로 3명 자르기 -> 양지은 제외
pre_prob = gbc_clf.predict_proba(X_test)[:, 1]
sor_list = sorted(pre_prob, reverse=True)
candidate = []
for i in sor_list:
  if i > 0.5: # 예측 확률이 0.5 이상이면 진선미라고 판단
    candidate.append(list(np.where(pre_prob == i)))

# 노래 살펴보기
jinsonmi_lst = []
for i in candidate:
  if df_test.iloc[i[0][0], 1] not in jinsonmi_lst:
    jinsonmi_lst.append(df_test.iloc[i[0][0], 1])

    if len(set(jinsonmi_lst)) < 4:
      print(df_test.iloc[i[0][0], 1])  

홍지윤
김태연
김다현


# (초기 시도) SGDClassifier와 RandomForestClassifier를 정찰병으로 사용하기
### 성능이 거의 50%...(심각)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42, verbose=2)
sgd_clf.fit(X_train, target)

-- Epoch 1
Norm: 40.08, NNZs: 13, Bias: 7.443175, T: 357, Avg. loss: 13.536854
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 43.46, NNZs: 14, Bias: -5.664824, T: 714, Avg. loss: 8.506848
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 36.14, NNZs: 15, Bias: 0.096825, T: 1071, Avg. loss: 8.942226
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 30.19, NNZs: 15, Bias: -4.162613, T: 1428, Avg. loss: 5.578455
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 31.13, NNZs: 15, Bias: -7.407340, T: 1785, Avg. loss: 6.642316
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 26.37, NNZs: 15, Bias: 2.793930, T: 2142, Avg. loss: 5.629577
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 16.40, NNZs: 15, Bias: -3.262955, T: 2499, Avg. loss: 4.513922
Total training time: 0.01 seconds.
-- Epoch 8
Norm: 24.01, NNZs: 15, Bias: 4.787836, T: 2856, Avg. loss: 4.450210
Total training time: 0.01 seconds.
-- Epoch 9
Norm: 17.68, NNZs: 15, Bias: -2.780556, T: 3213, Avg. loss: 4.432941
Total

SGDClassifier(random_state=42, verbose=2)

In [ ]:
cross_val_score(sgd_clf, X_train, target, cv=3, scoring='accuracy')

array([0.56302521, 0.58823529, 0.5210084 ])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
cross_val_score(forest_clf, X_train, target, cv=3, scoring='accuracy') 

array([0.5210084 , 0.50420168, 0.61344538])

# Pycaret 사용해서 각종 모델 성능 비교하기

In [ ]:
!pip install pycaret

     |████████████████████████████████| 266kB 6.6MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 215kB 9.0MB/s 
     |████████████████████████████████| 14.2MB 265kB/s 
     |████████████████████████████████| 1.7MB 41.1MB/s 
     |████████████████████████████████| 6.8MB 37.6MB/s 
     |████████████████████████████████| 245kB 43.3MB/s 
     |████████████████████████████████| 276kB 39.0MB/s 
     |████████████████████████████████| 102kB 7.7MB/s 
     |████████████████████████████████| 1.8MB 36.3MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 153kB 42.9MB/s 
     |████████████████████████████████| 163kB 42.0MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 1.1MB 37.8MB/s 
     |████████████████████████████████| 604kB 37.6MB/s 
     |████████████████████████████████| 3.1MB 12.5MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
   

In [ ]:
from pycaret.classification import *

In [ ]:
merged = pd.concat([X_train, target], axis=1)

In [ ]:
clf = setup(data = merged, target = 'target')

,Description,Value
0,session_id,4266
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(357, 16)"
5,Missing Values,False
6,Numeric Features,15
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
compare_models(sort = 'Accuracy')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6947,0.7100,0.4667,0.6163,0.4940,0.2972,0.3203,0.093
ada,Ada Boost Classifier,0.6505,0.6857,0.4444,0.5145,0.4526,0.2101,0.2243,0.113
dt,Decision Tree Classifier,0.6468,0.6296,0.5667,0.5029,0.5213,0.2478,0.2609,0.020
lr,Logistic Regression,0.6345,0.5211,0.0667,0.3417,0.1069,0.0279,0.0460,0.314
rf,Random Forest Classifier,0.6267,0.6461,0.3667,0.4680,0.3933,0.1432,0.1475,0.511
ridge,Ridge Classifier,0.6185,0.0000,0.0667,0.2533,0.0999,-0.0018,-0.0008,0.020
lda,Linear Discriminant Analysis,0.6185,0.5349,0.0778,0.2700,0.1151,0.0037,0.0074,0.022
lightgbm,Light Gradient Boosting Machine,0.6107,0.6222,0.3667,0.4576,0.3965,0.1217,0.1242,0.041
qda,Quadratic Discriminant Analysis,0.5983,0.5398,0.3333,0.4569,0.3354,0.0832,0.1024,0.023
svm,SVM - Linear Kernel,0.5867,0.0000,0.3556,0.3503,0.3188,0.0719,0.0907,0.020


# GBC가 accuracy 측면에서 가장 좋은 성능을 보여주었기 때문에 주 모델로 선택

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
      max_depth=5, random_state=0)
gbc_clf.fit(X_train, target)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
# pycaret에서 나온 accuracy와 너무 많은 차이가 남...
cross_val_score(gbc_clf, X_train, target, cv=3, scoring='accuracy')

array([0.54621849, 0.4789916 , 0.51260504])

# GBC에 대한 Gridsearch 진행

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
cv = KFold(n_splits=5, random_state=1)
param_grid={'learning_rate': [0.01, 0.025, 0.05, 0.1, 0.15],
            'max_depth':[5, 6, 8],
            'subsample':[0.5, 0.7, 0.8, 0.9, 1.0],
            #'min_samples_split': np.linspace(0.1, 0.5, 12),
            #'min_samples_leaf': np.linspace(0.1, 0.5, 12),
            'n_estimators':[50, 100, 150],
            'criterion': ['friedman_mse', 'mse', 'mae'],
            'max_features':['log2', 'sqrt'],
          }

grid_search = GridSearchCV(estimator = gbc_clf, param_grid = param_grid, 
                          cv = cv, n_jobs = -1, verbose = 2)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
# 평균 75%의 정확도로 나름 괜찮은 수준을 보이긴 함
grid_search.fit(X_train, train_target)
print(grid_search.score(X_train, train_target))
print(grid_search.best_params_)

Fitting 5 folds for each of 1350 candidates, totalling 6750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 495 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 912 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 2202 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 3071 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 4881 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 5650 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 6501 tasks      | elapsed: 17.6min


0.7513966480446927
{'criterion': 'mae', 'learning_rate': 0.01, 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 50, 'subsample': 0.7}


[Parallel(n_jobs=-1)]: Done 6750 out of 6750 | elapsed: 18.7min finished
